# 🚀 SadTalker – Stable Google Colab Notebook (Python 3.8, 2025)
⚠ GPU REQUIRED → Edit → Notebook settings → GPU
این نوت‌بوک تمام dependency ها را نصب می‌کند و آماده inference است.

In [ ]:
### 🔍 Check GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
### 🐍 Install Python 3.8 and pip safely
!sudo apt update
!sudo apt install -y python3.8 python3.8-distutils python3.8-venv
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python3.8 --version

# Install pip for Python 3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3.8 get-pip.py
!python3.8 -m pip install --upgrade pip
!python3.8 -m pip --version

In [ ]:
### 📥 Clone SadTalker Repo
print('Git clone project and install requirements...')
!git clone https://github.com/Winfredy/SadTalker &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH

In [ ]:
### 🔥 Install PyTorch compatible with CUDA 11.3
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
### 📦 Install requirements + facexlib + gfpgan + insightface
!apt update
!apt install -y ffmpeg
!python3.8 -m pip install -r requirements.txt
!python3.8 -m pip install ipywidgets matplotlib facexlib==0.3.0 insightface==0.7.3 gfpgan==1.3.8

In [ ]:
### 📥 Download pretrained models
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

## 👇 Select Image to Animate

In [ ]:
import ipywidgets as widgets
import glob
import matplotlib.pyplot as plt

print("Choose the image name to animate: (saved in folder 'examples/source_image/')")
img_list = glob.glob1('examples/source_image', '*.png')
img_list.sort()
img_list = [item.split('.')[0] for item in img_list]

default_head_name = widgets.Dropdown(options=img_list, value=img_list[0])

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plt.imshow(plt.imread(f'examples/source_image/{default_head_name.value}.png'))
        plt.axis('off')
        plt.show()

default_head_name.observe(on_change)
display(default_head_name)

plt.imshow(plt.imread(f'examples/source_image/{default_head_name.value}.png'))
plt.axis('off')
plt.show()

## ▶️ Run SadTalker (Generate Talking Video)

In [ ]:
img = f'examples/source_image/{default_head_name.value}.png'
print('Selected image:', img)

!python3.8 inference.py \
    --driven_audio ./examples/driven_audio/deyu.wav \
    --source_image "$img" \
    --result_dir ./results \
    --still --preprocess full --enhancer gfpgan